In [1]:
import torch 
from torch.utils import data 
from torch.autograd import Variable 
import torchvision
from torchvision.datasets import mnist
import matplotlib.pyplot as plt

In [2]:
from datascience import Table
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('fivethirtyeight')
import json
import pandas as pd

In [3]:
training = pd.read_csv('warframe_data.csv')
Train=Table.from_df(training)
Train

time,volume,min_price,max_price,avg_price,median_price,warframe,available,ducats,release,hot,date,weekend
2019/1/1,69,56,80,68,70,Banshee,4,260,0,1,1,0
2019/1/2,74,65,80,72.5,74,Banshee,4,260,0,1,2,0
2019/1/3,61,61,85,73,75,Banshee,4,260,0,1,3,0
2019/1/4,50,61,85,73,75,Banshee,4,260,0,1,4,0
2019/1/5,55,65,80,72.5,73.5,Banshee,4,260,0,1,5,1
2019/1/6,72,60,80,70,70,Banshee,4,260,0,1,6,1
2019/1/7,56,65,85,75,75,Banshee,4,260,0,1,7,0
2019/1/8,59,63,85,74,75,Banshee,4,260,0,1,8,0
2019/1/9,37,72,80,76,79.5,Banshee,4,260,0,1,9,0
2019/1/10,38,62,84,73,75,Banshee,4,260,0,1,10,0


In [175]:
features=[]
for i in range(1335):
    feature1=[]
    feature1.append(Train.row(i)[7])
    feature1.append(Train.row(i)[8]/100)
    feature1.append(Train.row(i)[9])
    feature1.append(Train.row(i)[10])
    feature1.append(Train.row(i)[11]/10)
    feature1.append(Train.row(i)[12])
    features.append(feature1)
features[:5]

[[4, 2.6, 0, 1, 0.1, 0],
 [4, 2.6, 0, 1, 0.2, 0],
 [4, 2.6, 0, 1, 0.3, 0],
 [4, 2.6, 0, 1, 0.4, 0],
 [4, 2.6, 0, 1, 0.5, 1]]

In [176]:
labels=[]
for i in range(1335):
    label=[]
    label.append(Train.row(i)[1])
    label.append(Train.row(i)[2])
    label.append(Train.row(i)[3])
    label.append(Train.row(i)[4])
    labels.append(label)
labels[:5]

[[69, 56, 80, 68.0],
 [74, 65, 80, 72.5],
 [61, 61, 85, 73.0],
 [50, 61, 85, 73.0],
 [55, 65, 80, 72.5]]

In [177]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

In [213]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(6,24) 
        self.fc2 = nn.Linear(24,12)
        self.fc3 = nn.Linear(12,4)
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [214]:
class MyDataset(Dataset):
    def __init__(self,labels,features):
        super(MyDataset,self).__init__()
        self.labels=labels
        self.features=features
    def __len__(self):
        return self.features.shape[0]
    def __getitem__(self,idx):
        feature=self.features[idx]
        label=self.labels[idx]
        return {'feature':feature, 'label':label}

In [215]:
def train_epoch(loader):
    total_loss=0.0
    for i,data in enumerate(loader):
        featurest=data['feature'].float()
        labelst=data['label'].float()
        optimizer.zero_grad()
        out=mnet(featurest)
        loss=loss_fn(out,labelst)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
    print ('loss', total_loss/i)

In [239]:
mnet=Net()
loss_fn=torch.nn.MSELoss()
optimizer=torch.optim.SGD(mnet.parameters(),lr=0.0001,momentum=0.01,weight_decay=1e-8)

dataset=MyDataset(np.asarray(labels),np.asarray(features))
load=DataLoader(dataset,shuffle=True,batch_size=100)

for epoch in range(1000):
    train_epoch(load)

loss 31054.17578125
loss 30537.702674278848
loss 21208.9208984375
loss 10199.054612379809
loss 6617.003079927885
loss 4120.304912860577
loss 3067.138653094952
loss 2708.5537391075723
loss 2525.2543006310098


In [249]:
k=562
state=features[k]
innput=Variable(torch.FloatTensor(state))
output=mnet(innput).detach().numpy()
print(output)
print(labels[k])
Train.row(k)

[171.18152 133.69408 160.19861 146.79666]
[169, 138, 164, 151.0]


Row(time='2019/1/29', volume=169, min_price=138, max_price=164, avg_price=151.0, median_price=150.0, warframe='Mesa', available =3, ducats=260, release=1, hot=3, date=29, weekend=0)

In [250]:
torch.save(mnet.state_dict(),'DNNmodel.pth')